In [264]:
import pandas as pd
import seaborn as sns
from numpy import *

from sklearn.preprocessing import OneHotEncoder
from category_encoders.target_encoder import TargetEncoder

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [505]:
df = pd.read_csv('ya_climate.csv', sep=';', decimal='.')

In [506]:
df.columns = list(map(lambda x: x.lower().replace(' ', '_'), df.columns))

In [507]:
df = df.drop_duplicates()

In [508]:
col = "температура_воздуха_на_улице"
average_col = "среднемесячная_температура_на_улице"
df[col] = df[col].fillna(df.groupby(["город", "время_года"])[average_col].transform("mean"))

In [509]:
col = "среднемесячная_температура_на_улице"
mean_temp = df.groupby("город")[col].mean()
df.loc[df[col] > 100, col] = df["город"].map(mean_temp)
df[col] = df[col].apply(lambda x: x if x > 50 else x)

In [ ]:
d = df[df['страна'] == 'США']['температура_воздуха_в_помещении']
d[d > 70] = round((d[d > 70] - 32) * 5 / 9, 4)
df.loc[df['страна'] == 'США', 'температура_воздуха_в_помещении'] = d

In [511]:
df2 = df[['год', 'климат', 'способ_охлаждения',
        'способ_обогрева',
          'ощущение_температуры', 'температура_воздуха_в_помещении',
        'предпочтительное_изменение_температуры', 'окно', 'двери',
                'среднемесячная_температура_на_улице']]

In [512]:
df2.loc[df2['климат'] == 'Cубтроп океанич', 'климат'] = 'Cубтропический океанический'

In [513]:
climate = df2.groupby(['климат'])['температура_воздуха_в_помещении'].agg(['mean']).reset_index() 
# среднее значение температуры в каждом климате

In [ ]:
df2.климат = df2.климат.map(lambda x: climate[climate['климат'] == x]['mean'].iloc[0])

In [515]:
ohe1 = OneHotEncoder()
o = ohe1.fit_transform(df2['способ_охлаждения'].to_numpy().reshape(-1, 1))

o = pd.DataFrame(o.toarray())

df2.insert(0, 'Кондиционирование', o.iloc[:, 0])
df2.insert(0, 'Смешанный', o.iloc[:, 1])
df2.insert(0, 'Вентиляция', o.iloc[:, 2])

In [516]:
df2 = df2.drop('способ_охлаждения', axis=1)

In [517]:
df2.loc[df2['способ_обогрева'].isna(), 'способ_обогрева'] = 0
df2.loc[df2['способ_обогрева'] == 'Механическое отопление', 'способ_обогрева'] = 1

In [518]:
col = 'предпочтительное_изменение_температуры'

In [519]:
df2.loc[df2[col] == 'Холодн', col] = 'Холоднее'
df2.loc[df2[col] == 'Тепле', col] = 'Теплее'

In [520]:
df2.loc[df2[col] == 'Без изменений', col] = 0
df2.loc[df2[col] == 'Теплее', col] = 1
df2.loc[df2[col] == 'Холоднее', col] = -1

In [521]:
df2['окно'] = df2['окно'].fillna(3).astype('category')
df2['двери'] = df2['двери'].fillna(3).astype('category')

In [522]:
Y = df2['температура_воздуха_в_помещении']
X = df2.drop('температура_воздуха_в_помещении', axis=1)

In [523]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=0, test_size=0.2)

In [524]:
target_encoder_window = TargetEncoder()
target_encoder_window.fit(X_train['окно'], y_train)
target_encoded_window = target_encoder_window.transform(X_train['окно'])
X_train['окно'] = target_encoded_window.astype('category')

In [525]:
target_encoder_door = TargetEncoder()
target_encoder_door.fit(X_train['двери'], y_train)
target_encoded_door = target_encoder_door.transform(X_train['двери'])
X_train['двери'] = target_encoded_door.astype('category')

In [526]:
target_encoded_door = target_encoder_door.transform(X_test['двери'])
X_test['двери'] = target_encoded_door.astype('category')

In [527]:
target_encoded_window = target_encoder_window.transform(X_test['окно'])
X_test['окно'] = target_encoded_window.astype('category')

In [528]:
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [529]:
print(model.score(X_train, y_train))
print(model.score(X_test, y_test))

0.45065370809771355
0.38432884357914776
